<h2>Titanic survival prediction<h2\>

<h2>import library  and needed data <h2\> 

In [1]:
import pandas as pd
import numpy as np


In [2]:
data=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
test_ids=test["PassengerId"]
data.head(6)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q


<h3>Removing useless data:<h3\>
    <li>Ticket<li\>
    <li>Cabin<li\>
    <li>Name<li\>   
    <li>PassengerId<li\>

In [3]:
data=data.drop(['PassengerId','Ticket','Cabin','Name'],axis=1)
test=test.drop(['PassengerId','Ticket','Cabin','Name'],axis=1)

<h3>correction of the targeted columns for data  <h3\>

In [4]:
cols1=["SibSp","Parch","Fare"]
for col in cols1 : 
    data[col].fillna(data[col].median(),inplace=True)
cols2=["SibSp","Parch","Fare","Age"]  
for col in cols2 : 
    test[col].fillna(data[col].median(),inplace=True)
    

In [5]:
x1,x2,x3=[],[],[]
index3=[]
index1=[]
index2=[]
for e in range(data.shape[0]):
        if data["Pclass"][e]==3: 
                x3.append(data["Age"][e])
                index3.append(e)
for e in range(data.shape[0]):
        if data["Pclass"][e]==2: 
                x2.append(data["Age"][e])
                index2.append(e)                

for e in range(data.shape[0]):
        if data["Pclass"][e]==1: 
                x1.append(data["Age"][e])
                index1.append(e)                
                
                
df_Ages3=pd.DataFrame({'Age3': x3,'index3':index3})  
df_Ages2=pd.DataFrame({'Age2':x2,'index2':index2})
df_Ages1=pd.DataFrame({'Age1':x1,'index1':index1})
df_Ages3['Age3'].fillna(df_Ages3['Age3'].mean(),inplace=True) 
df_Ages2['Age2'].fillna(df_Ages2['Age2'].mean(),inplace=True)
df_Ages1['Age1'].fillna(df_Ages1['Age1'].mean(),inplace=True)


In [6]:
array1=df_Ages1['Age1'].to_numpy()
array2=df_Ages2['Age2'].to_numpy()         
array3=df_Ages3['Age3; '].to_numpy()
a,b,c=0,0,0

#corection for ages for first class
for e in range(data.shape[0]):
    if data['Pclass'][e]==1 : 
        a=index1.index(e)
        data.loc[e, 'Age'] =array1[a]
#corection for ages for second class        
for e in range(data.shape[0]):
    if data['Pclass'][e]==2 : 
        a=index2.index(e)
        data.loc[e, 'Age'] =array2[a]
#corection for ages for third class        
for e in range(data.shape[0]):
    if data['Pclass'][e]==3 : 
        a=index3.index(e)
        data.loc[e, 'Age'] =array3[a]  
        
    

In [7]:
data.head(6)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.00000,1,0,7.2500,S
1,1,1,female,38.00000,1,0,71.2833,C
2,1,3,female,26.00000,0,0,7.9250,S
3,1,1,female,35.00000,1,0,53.1000,S
4,0,3,male,35.00000,0,0,8.0500,S
5,0,3,male,25.14062,0,0,8.4583,Q


In [8]:
from sklearn import preprocessing 
le=preprocessing.LabelEncoder()
cols=['Sex','Embarked']
for col  in cols :
    data[col]=le.fit_transform(data[col])
    test[col]=le.fit_transform(test[col])
    print(le.classes_)

['female' 'male']
['C' 'Q' 'S']


In [9]:
data.head(3)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [11]:
y=data["Survived"]
x=data.drop("Survived",axis=1)
x_train,x_val ,y_train,y_val =train_test_split(x,y,test_size=0.2,random_state=42)

In [12]:
clf=LogisticRegression(random_state = 0,max_iter=1000).fit(x_train,y_train)

In [13]:
predictions=clf.predict(x_val)
from sklearn.metrics import accuracy_score 
accuracy_score(y_val,predictions)

0.8156424581005587

In [14]:
submission_preds=clf.predict(test)

In [15]:
df=pd.DataFrame({"passengerId":test_ids.values,
                "Survived":submission_preds,
                })

In [16]:
df.to_csv("submission.csv",index=False)